### Download data

In [ ]:
! gdown 1L1v_wwa8GwEGUy39Xls1JF2VoU0hSDJ7

In [ ]:
! pip install -r requirements.txt

In [ ]:
! pip install transformers

In [ ]:
! pip install adapter-transformers

In [ ]:
! ls /content

In [ ]:
! mkdir /content/drive/MyDrive/MrEnImageCaptioning

In [ ]:
! cp -r *.py /content/drive/MyDrive/MrEnImageCaptioning

### Russian Data Creation

In [ ]:
import json

with open("eng_predicted_captions.json", "r") as stream:
    data = json.loads(stream.read())

In [ ]:
len(data)

In [ ]:
print(data[0])

In [ ]:
eval_data = []
for item in data:
    eval_data.append({
        "translation": {"en": item["hypothesis"], "mr": item["hypothesis"]}
    })

In [ ]:
eval_data[0]

In [ ]:
with open("en-to-mr-test.json", "w", encoding="utf-8") as stream:
    for jsonl_dict in eval_data:
        stream.write(json.dumps(jsonl_dict, ensure_ascii=False).encode('utf8').decode()+"\n")

In [ ]:
combined_mr_data = []

with open("mr_en_train.json", "r") as stream_in:
    with open("en-to-mr-train.json", "w", encoding="utf-8") as stream_out:
        for line in stream_in.readlines():
            stream_out.write(line)

with open("mr_en_dev.json", "r") as stream_in:
    with open("en-to-mr-train.json", "a", encoding="utf-8") as stream_out:
        for line in stream_in.readlines():
            stream_out.write(line)

with open("mr_en_test.json", "r") as stream_in:
    with open("en-to-mr-train.json", "a", encoding="utf-8") as stream_out:
        for line in stream_in.readlines():
            stream_out.write(line)

### Adapter fine-tuning

In [ ]:
! python /content/finetune_adapter.py \
    --model_name_or_path "facebook/mbart-large-50-many-to-many-mmt" \
    --train_adapter \
    --do_train \
    --do_eval \
    --do_predict \
    --evaluation_strategy "steps" \
    --save_total_limit 1 \
    --fp16 \
    --eval_steps 5000 \
    --train_file en-to-mr-train.json \
    --validation_file en-to-mr-test.json \
    --test_file en-to-mr-test.json \
    --source_lang en_XX \
    --target_lang mr_IN \
    --output_dir /content/mbart/en-mr \
    --per_device_train_batch_size=8 \
    --per_device_eval_batch_size=8 \
    --overwrite_output_dir \
    --num_train_epochs 3 \
    --predict_with_generate

In [ ]:
from google.colab import files
files.download("mbart/en-mr/generated_predictions.txt")

### Formatting Predictions

In [ ]:
with open("ru20k_predicted_captions.json", "r") as stream:
  ru_preds = json.loads(stream.read())

ru_test = []
with open("mbart/en-mr/generated_predictions.txt", "r") as stream:
  for line in stream.readlines():
    ru_test.append(line.strip())

assert len(ru_preds) == len(ru_test)

for idx, (gt, pred) in enumerate(zip(ru_preds, ru_test)):
  ru_preds[idx]["hypothesis"] = pred

with open("mBART50_mr_preds.json", "w", encoding="utf-8") as stream:
  stream.write(json.dumps(ru_preds, ensure_ascii=False).encode('utf8').decode())

files.download("mBART50_mr_preds.json")